## Smart RAG
- Know what you know and what you don't know.


In [58]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25

In [59]:

%load_ext dotenv
%dotenv
# UPSTAGE_API_KEY

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [60]:
import warnings

warnings.filterwarnings("ignore")

In [73]:
from langchain_upstage import UpstageLayoutAnalysisLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings


# Load doc
layzer = UpstageLayoutAnalysisLoader("solar_paper.pdf", output_type="html")
# For improved memory efficiency, consider using the lazy_load method to load documents page by page.
docs = layzer.load()  # or layzer.lazy_load()

In [74]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage


llm = ChatUpstage()

prompt_template = PromptTemplate.from_template(
    """
    Please provide most correct answer from the following context. 
    If the answer is not present in the context, please write "The information is not present in the context."

    ---
    Question: {question}
    ---
    Context: {context}
    """
)
chain = prompt_template | llm | StrOutputParser()

In [75]:
chain.invoke({"question": "What is DUS?", "context": docs})

'DUS stands for Depth Up-Scaling.'

In [77]:
chain.invoke({"question": "How to get to Seoul from SF", "context": docs})

'To get to Seoul from San Francisco (SF), you can take a flight from San Francisco International Airport (SFO) to Incheon International Airport (ICN) in Seoul. There are several airlines that offer direct flights from SFO to ICN, including Korean Air, Asiana Airlines, and United Airlines. The flight duration is typically around 12-14 hours, depending on the stopovers and the airline. You can also choose to take a connecting flight through other cities like Tokyo, Beijing, or Hong Kong, but it would take longer.'

In [78]:
# RAG or Search?
def is_in(question, context):
    is_in_conetxt = """As a helpful assistant, 
please use your best judgment to determine if the answer to the question is within the given context. 
If the answer is present in the context, please respond with "yes". 
If not, please respond with "no". 
Only provide "yes" or "no" and avoid including any additional information. 
Please do your best. Here is the question and the context.:
---
CONTEXT: {context}
---
QUESTION: {question}
---
OUTPUT (yes or no):"""

    is_in_prompt = PromptTemplate.from_template(is_in_conetxt)
    chain = is_in_prompt | ChatUpstage() | StrOutputParser()

    response = chain.invoke({"context": context, "question": question})
    print(response)
    return response.lower().startswith("yes")

In [79]:
is_in("How to get to Seoul from SF", docs)

no


False

In [82]:
is_in("What is DUS?", docs)

yes


True

In [85]:
# Smart RAG, Self-Improving RAG
import os
from tavily import TavilyClient


def smart_rag(question):
    if not is_in(question, docs):
        print("Searching in tavily")
        tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])
        context = tavily.search(query=question)

    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"context": docs, "question": question})

In [87]:
smart_rag("What is DUS?")

yes


UnboundLocalError: local variable 'context' referenced before assignment

In [71]:
smart_rag("How to get to Seoul from SF?")

no
Searching in tavily


"To get to Seoul from SF, you can take the following steps:\n\n1. Take the BART from Civic Center / UN Plaza to Milpitas.\n2. Fly from San Jose (SJC) to Incheon (ICN) SJC - ICN.\n3. Take the train from Incheon Int'l Airport T1 to Seoul Station.\n\nThe estimated cost for this route is between ₩615,005 and ₩1221,200."

# Excercise
TBA